# ECS709 - Introduction to Computer Vision
## Coursework

In [ ]:
!pip install numpy

In [ ]:
!pip install opencv-python

In [ ]:
!pip install matplotlib

In [ ]:
!pip install pandas

In [ ]:
!pip install opencv-python --no-binary opencv-python

In [ ]:
!pip install --upgrade pip setuptools

In [ ]:
!pip install opencv-python-headless

In [ ]:
!pip install scipy

## 1) Transformations.
<br>Rotation, translation and skew are useful operations for matching, tracking, and data augmentation.

#### a) Write a function that takes as input an image I, rotates it by an angle θ1 and horizontally skews it by an angle, θ2. Write the matrix formulation for image rotation R(.) and skewing S(.). Define all the variables. Note that the origin of the coordinate system of the programming environment you use might be different from the one shown in the lectures.

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def rotate_with_interpolation(main_img, angle_of_rotation, rotation_point):
    angle_of_rotation = np.radians(angle_of_rotation)
    h, w = main_img.shape[:2]
    rotation_point_x, rotation_point_y = rotation_point

    # Calculate the new dimensions for the rotated image
    cos_theta = np.abs(np.cos(angle_of_rotation))
    sin_theta = np.abs(np.sin(angle_of_rotation))
    new_w = int(h * sin_theta + w * cos_theta)
    new_h = int(h * cos_theta + w * sin_theta)

    new_img = np.zeros((new_h, new_w, 3), dtype=np.uint8)

    for new_y in range(new_h):
        for new_x in range(new_w):
            x = (new_x - new_w // 2) * np.cos(angle_of_rotation) + (new_y - new_h // 2) * np.sin(angle_of_rotation) + rotation_point_x
            y = -(new_x - new_w // 2) * np.sin(angle_of_rotation) + (new_y - new_h // 2) * np.cos(angle_of_rotation) + rotation_point_y

            if 0 <= x < w - 1 and 0 <= y < h - 1:
                x0 = int(x)
                y0 = int(y)
                x1 = x0 + 1
                y1 = y0 + 1

                # Bilinear interpolation
                dx = x - x0
                dy = y - y0

                for channel in range(3):  # For RGB channels
                    new_img[new_y, new_x, channel] = (
                        (1 - dx) * (1 - dy) * main_img[y0, x0, channel] +
                        dx * (1 - dy) * main_img[y0, x1, channel] +
                        (1 - dx) * dy * main_img[y1, x0, channel] +
                        dx * dy * main_img[y1, x1, channel]
                    )

    # Find the bounding box of the rotated image
    non_zero_indices = np.nonzero(new_img)
    min_y, max_y = np.min(non_zero_indices[0]), np.max(non_zero_indices[0])
    min_x, max_x = np.min(non_zero_indices[1]), np.max(non_zero_indices[1])

    # Calculate the dimensions of the rotated image without the background frame
    rotated_h = max_y - min_y + 1
    rotated_w = max_x - min_x + 1

    return new_img[min_y:max_y+1, min_x:max_x+1, :]

img = cv2.imread('cat.jpg')
height, width = img.shape[:2]
rotation_point = (width // 2, height // 2)
angle = int(input("Enter angle in degrees:"))  # Convert angle to radians

rotated_img = rotate_with_interpolation(img, angle, rotation_point)

plt.imshow(rotated_img)
plt.show()

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

def shear_with_interpolation(main_img, shear_angle):
    h, w = main_img.shape[:2]

    shear_matrix = np.array([
        [1, np.tan(np.radians(shear_angle))],
        [0, 1]
    ])

    new_w = int(w + abs(np.tan(np.radians(shear_angle))) * h)
    new_h = h

    new_img = np.zeros((new_h, new_w, 3), dtype=np.uint8)

    for new_y in range(new_h):
        for new_x in range(new_w):
            original_coords = np.dot(np.linalg.inv(shear_matrix), np.array([new_x, new_y]))
            x = original_coords[0]
            y = original_coords[1]

            if 0 <= x < w - 1 and 0 <= y < h - 1:
                x0 = int(x)
                y0 = int(y)
                x1 = x0 + 1
                y1 = y0 + 1

                # Bilinear interpolation
                dx = x - x0
                dy = y - y0

                for channel in range(3):  # For RGB channels
                    new_img[new_y, new_x, channel] = (
                        (1 - dx) * (1 - dy) * main_img[y0, x0, channel] +
                        dx * (1 - dy) * main_img[y0, x1, channel] +
                        (1 - dx) * dy * main_img[y1, x0, channel] +
                        dx * dy * main_img[y1, x1, channel]
                    )

    return new_img

img = cv2.imread('cat.jpg')
height, width = img.shape[:2]

shear_angle = float(input("Enter shear angle in degrees: "))

sheared_img = shear_with_interpolation(img, shear_angle)

plt.imshow(sheared_img)
plt.show()


In [ ]:
img = cv2.imread('cat.jpg')
height, width = img.shape[:2]
rotation_point = (width // 2, height // 2)
angle = np.radians(int(input("Enter rotation angle in degrees: ")))  # Convert angle to radians
shear_angle = float(input("Enter shear angle in degrees: "))
rotated_img = rotate_with_interpolation(img, angle, rotation_point)
sheared_img = shear_with_interpolation(rotated_img, shear_angle)

plt.imshow(sheared_img)
plt.show()

#### b) Create an image that contains your name written in Arial, point 72, capital letters. Rotate clockwise the image you created by 30, 60, 120 and -50 degrees. Skew the same image by 10, 40 and 60 degrees. Complete the process so that all the pixels have a value. Discuss in the report the advantages and disadvantages of different approaches.

In [ ]:
from matplotlib import pyplot as plt

text = "Nikita"
font = {'family': 'Arial', 'weight': 'normal', 'size': 72}

plt.text(0.5, 0.5, text, ha='center', va='center', **font)
# Remove the x-axis
plt.gca().xaxis.set_visible(False)

# Remove the y-axis
plt.gca().yaxis.set_visible(False)
plt.savefig('my_name.jpg')
plt.show()

In [ ]:
img = cv2.imread('my_name.jpg')
rotation_angle = [30, 60, 120, -50]
shear_angle = [10, 40, 60]
for i in rotation_angle:
    height, width = img.shape[:2]
    rotation_point = (width // 2, height // 2)
    rotated_img = rotate_with_interpolation(img, i, rotation_point)
    plt.imshow(rotated_img)
    plt.show()

for j in shear_angle:
    sheared_img = shear_with_interpolation(img,j,)
    plt.imshow(sheared_img)
    plt.show()

#### c) Analyse the results when you change the order of the two operators: R(S(I)) and S(R(I)).
#### i) Rotate the image by θ1 = 20 clockwise and then skew the result by θ2 = 50. 
#### ii) Skew the image by θ2 = 50 and then rotate the result by θ1 = 20 clockwise.
#### Are the results of (i) and (ii) the same? Why?

In [ ]:
img = cv2.imread('cat.jpg')
height, width = img.shape[:2]
rotation_point = (width // 2, height // 2)
angle = 20  # Convert angle to radians
shear_angle = 50
rotated_img = rotate_with_interpolation(img, angle, rotation_point)
sheared_img = shear_with_interpolation(rotated_img, shear_angle)

plt.imshow(sheared_img)
plt.show()

In [ ]:
img = cv2.imread('cat.jpg')
height, width = img.shape[:2]
rotation_point = (width // 2, height // 2)
shear_angle = 50
angle = 20  # Convert angle to radians
sheared_img = shear_with_interpolation(rotated_img, shear_angle)
rotated_img = rotate_with_interpolation(img, angle, rotation_point)


plt.imshow(sheared_img)
plt.show()

## 2) Convolution. (Use Dataset A)

#### Convolution provides a way of multiplying two arrays to produce a third array. Depending on the designed filter and the intended effect, the kernel can be a matrix of dimensions, for example, 3x3, 5x5 or 7x7.

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt
import cv2
import os

### Convolution

In [ ]:
def convolution_grayscale(input_image, kernel):
    # height and width of input image
    input_image_height, input_image_width = input_image.shape
    # height and width of the kernel
    kernel_height, kernel_width = kernel.shape

    # Calculating the padding required around the image
    pad_height = kernel_height // 2
    pad_width = kernel_width // 2

    # Padding the image
    padded_input_image = np.pad(input_image, ((pad_height, pad_height), (pad_width, pad_width)), mode='constant')

    # Create an empty output_image
    output_image = np.zeros_like(input_image)

    # Convolution operation formula -> h(n) = f(n) * g(n)
    for i in range(input_image_height):
        for j in range(input_image_width):
            output_image[i, j] = np.sum(padded_input_image[i:i + kernel_height, j:j + kernel_width] * kernel)

    # Normalize the output_image to achieve smoother results
    output_image = (output_image - np.min(output_image)) / (np.max(output_image) - np.min(output_image)) * 255

    return output_image.astype(np.uint8)


In [ ]:
import numpy as np
from scipy.signal import convolve2d
import matplotlib.pyplot as plt
from PIL import Image

# Function for image convolution
def apply_convolution(image, kernel):
    return convolve2d(image, kernel, mode='same', boundary='symm')

## Display and save function 

In [ ]:
def display_and_save_image(kernel,path = None,name_of_folder = "images"):
    if path: 
        folder_path = path
    else:
        # Specify the folder containing the images
        folder_path = 'Dataset/DatasetA/'

    # Get the absolute path for the folder
    folder_path = os.path.abspath(folder_path)

    # List all image files in the folder with absolute paths
    image_files = [os.path.abspath(os.path.join(folder_path, f)) for f in os.listdir(folder_path) if f.endswith('.jpg')]

    # Create a directory to save the resulting images
    name_of_folder = name_of_folder
    output_directory = os.path.join('Dataset/DatasetA/', name_of_folder)
    os.makedirs(output_directory, exist_ok=True)

    # Iterate through the images, apply convolution, and save the resulting images
    for image_file in image_files:
        # Load the image
        input_image = cv2.imread(image_file, cv2.IMREAD_GRAYSCALE)

        # Perform convolution using the averaging kernel
        result_image = apply_convolution(input_image, kernel)

        # Display the original and convolved images
        plt.figure(figsize=(10, 5))

        plt.subplot(1, 2, 1)
        plt.imshow(input_image, cmap='gray')
        plt.title('Original Image')
        plt.axis('off')

        plt.subplot(1, 2, 2)
        plt.imshow(result_image, cmap='gray')
        plt.title('Convolved Image')
        plt.axis('off')

        plt.show()

        # Define the output path for the convoluted image
        output_filename = os.path.splitext(os.path.basename(image_file))[0] + '_convoluted.jpg'
        output_image_path = os.path.join(output_directory, output_filename)

        # Save the resulting image with the full output path
        cv2.imwrite(output_image_path, result_image)
        


#### a) Code a function that takes an input image, performs convolution with a given kernel, and returns the resulting image.

In [ ]:
kernel = np.array([[0, 0, 0],
                   [0, 1, 0],
                   [0, 0, 0]]) 

display_and_save_image(kernel,'Dataset/DatasetA/','normal_kernel')

#### b) Design a convolution kernel that computes, for each pixel, the average intensity value in a 3x3 region. Use this kernel and the filtering function above, and save the resulting image.

In [ ]:
# Define the 3x3 averaging kernel
kernel = np.array([[1/9, 1/9, 1/9],
                   [1/9, 1/9, 1/9],
                   [1/9, 1/9, 1/9]])

display_and_save_image(kernel,'Dataset/DatasetA/','average_kernel')

#### c) Use the kernels provided below, apply the filtering function and save the resulting images. Comment on the effect of each kernel.
kernel A
1 2 1
2 4 2
1 2 1
kernel B
0 1 0
1 -4 1
0 1 0

### Applying filtering function(Kernel A) on RGB Image

In [ ]:
kernel_A = np.array([[1, 2, 1],
                   [2, 4, 2],
                   [1, 2, 1]])

kernel_A = kernel_A / np.sum(np.abs(kernel_A))

display_and_save_image(kernel_A,'Dataset/DatasetA/average_kernel','kernel_A')

### Applying filtering function(Kernel B) on Grayscale Image

In [ ]:
kernel_B = np.array([[0, 1, 0],
                   [1, -4, 1],
                   [0, 1, 0]])

kernel_B = kernel_B / np.sum(np.abs(kernel_B))
display_and_save_image(kernel_B,'Dataset/DatasetA/average_kernel','kernel_B')

#### d)Use the filtering function for the following filtering operations: (i) A followed by A; (ii) A followed by B; (iii) B followed by A. Comment the results.

In [ ]:
def display_and_save_for_two_kernel_image(kernel_1,kernel_2):
    
      # Specify the folder containing the images
    folder_path = 'Dataset/DatasetA/'

    # Get the absolute path for the folder
    folder_path = os.path.abspath(folder_path)

    # List all image files in the folder with absolute paths
    image_files = [os.path.abspath(os.path.join(folder_path, f)) for f in os.listdir(folder_path) if f.endswith('.jpg')]

    # Create a directory to save the resulting images
    output_directory = os.path.join(folder_path, 'averaged_images')
    os.makedirs(output_directory, exist_ok=True)

    # Iterate through the images, apply convolution, and save the resulting images
    for image_file in image_files:
        # Load the image
        input_image = cv2.imread(image_file, cv2.IMREAD_GRAYSCALE)
        kernel = np.array([[1/9, 1/9, 1/9],
                   [1/9, 1/9, 1/9],
                   [1/9, 1/9, 1/9]])

        input_input = apply_convolution(input_image,kernel)
        # Perform convolution using the first kernel
        kernel_1 = kernel_1 / np.sum(np.abs(kernel_1))
        result_image_kernel_1 = apply_convolution(input_image, kernel_1)
        kernel_2 = kernel_2 / np.sum(np.abs(kernel_2))
        # Perform convolution using the second kernel
        result_of_two_kernel_image = apply_convolution(result_image_kernel_1, kernel_2)

        # Display the original and convolved images
        plt.figure(figsize=(10, 5))

        plt.subplot(1, 2, 1)
        plt.imshow(input_image, cmap='gray')
        plt.title('Original Image')
        plt.axis('off')

        plt.subplot(1, 2, 2)
        plt.imshow(result_of_two_kernel_image, cmap='gray')
        plt.title('Convolved Image (Two Kernels)')
        plt.axis('off')

        plt.show()

        # Define the output path for the convoluted image
        output_filename = os.path.splitext(os.path.basename(image_file))[0] + '_convoluted.jpg'
        output_image_path = os.path.join(output_directory, output_filename)

        # Save the resulting image with the full output path
        cv2.imwrite(output_image_path, result_of_two_kernel_image)
        plt.show()
       

**(i) A followed by A**

In [ ]:
display_and_save_for_two_kernel_image(kernel_A,kernel_A)

**(ii) A followed by B**

In [ ]:
display_and_save_for_two_kernel_image(kernel_A,kernel_B)

**(iii) B followed by A.**

In [ ]:
display_and_save_for_two_kernel_image(kernel_B,kernel_A)

### 3) Video Segmentation. (Use Dataset B)
A colour histogram h(.) can be generated by counting how many times each colour occurs in an image.
Histogram intersection can be used to match a pair of histograms. Given a pair of histograms, e.g., of an 
input image I and a model M, each containing n bins, the intersection of the histograms is defined as 
∑ min[h(Ij), h(Mj)]
n
j=1
. 

#### a)Write a histogram function that returns the colour histogram of an input image. Visualize the histogram  and save the corresponding figure. For a given video sequence, use the above function to construct the histogram of each frame.

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

def color_histogram(image):
    # Convert the image from BGR to RGB
    img_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Calculate the histogram for each color channel
    hist_r = cv2.calcHist([img_rgb], [0], None, [256], [0, 256])  # Red
    hist_g = cv2.calcHist([img_rgb], [1], None, [256], [0, 256])  # Green
    hist_b = cv2.calcHist([img_rgb], [2], None, [256], [0, 256])  # Blue

    # Normalize the histograms
    hist_r = hist_r / np.sum(hist_r)
    hist_g = hist_g / np.sum(hist_g)
    hist_b = hist_b / np.sum(hist_b)

    return hist_r, hist_g, hist_b

def plot_histogram(frame_lst):
    # Plot the histograms
    if len(frame_lst) == 1:
        hist_r, hist_g, hist_b = color_histogram(frame_lst[0])
        plt.plot(hist_r, color='red', label='Red', alpha=0.7)
        plt.plot(hist_g, color='green', label='Green', alpha=0.7)
        plt.plot(hist_b, color='blue', label='Blue', alpha=0.7)
        plt.title('Frame')
        plt.savefig(f'Dataset/histogram/color_histogram.png')
        plt.xlabel('Pixel Value')
        plt.ylabel('Frequency')
        plt.legend()
    else:
        if len(frame_lst) != 0:
            for i in range(0, len(frame_lst), 5):  # Increment by 6 frames
                fig, axs = plt.subplots(1, 5, figsize=(18, 5))
                for j in range(5):
                    if i + j < len(frame_lst):
                        hist_r, hist_g, hist_b = color_histogram(frame_lst[i + j])
                        axs[j].plot(hist_r, color='red', label='Red', alpha=0.7)
                        axs[j].plot(hist_g, color='green', label='Green', alpha=0.7)
                        axs[j].plot(hist_b, color='blue', label='Blue', alpha=0.7)
                        axs[j].set_title(f'Frame {i + j}')

                # Customize the plot for each row
                for ax in axs:
                    ax.set_xlabel('Pixel Value')
                    ax.set_ylabel('Frequency')
                    ax.legend()

                plt.tight_layout()

                # Save the figure for each row
                plt.savefig(f'Dataset/histogram/color_histogram_row_{i // 5}.png')

                # Show the plot for each row
                plt.show()

In [ ]:
plot_histogram([cv2.imread("test.jpg")])

In [ ]:
def process_video(video_path):

    capture = cv2.VideoCapture(video_path)

    if not capture.isOpened():
        print("Error: Could not open video.")
        return 
    frame_lst = []
    while True:
        ret,frame = capture.read()

        if not ret:
            break

        
        hist_r,hist_g,hist_b = color_histogram(frame)
        frame_lst.append(frame)
    plot_histogram(frame_lst)

    # Release the video capture object
    capture.release()
    return frame_lst


# Example usage for a video
video_path = 'Dataset/DatasetB.avi'
frame_lst = process_video(video_path)


In [ ]:
plt.imshow(frame_lst[8])

In [ ]:
plt.imshow(frame_lst[9])

In [ ]:
plt.imshow(frame_lst[100])

### b) Write a function that returns the value of the intersection of a pair of histograms. For a given video sequence, use the histogram intersection function to calculate the intersection between consecutive frames (e.g. between It and It+1, between It+1 and It+2 and so on). Find how to normalize the intersection. Does that change the results? Plot the intersection values over time and the normalised intersection values, and save the corresponding figures. Show and comment the figures in the report.

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
def calculate_histogram_intersection(histogram1, histogram2):
    # Normalize histograms
    histogram1_normalized = histogram1 / sum(histogram1)
    histogram2_normalized = histogram2 / sum(histogram2)

    # Calculate histogram intersection
    intersection = np.min(np.vstack((histogram1_normalized, histogram2_normalized)), axis=0)
    return sum(intersection)

In [ ]:
def calculate_frame_intersection(video_path):
    capture = cv2.VideoCapture(video_path)

    if not capture.isOpened():
        print("Error: Could not open video.")
        return

    frame_lst = []

    while True:
        ret, frame = capture.read()

        if not ret:
            break

        frame_lst.append(frame)

    # Release the video capture object
    capture.release()

    # Calculate intersection between consecutive frames
    intersection_values = []
    for i in range(len(frame_lst) - 1):
        histogram1 = cv2.calcHist([frame_lst[i]], [0], None, [256], [0, 256])
        histogram2 = cv2.calcHist([frame_lst[i+1]], [0], None, [256], [0, 256])
        intersection = calculate_histogram_intersection(histogram1, histogram2)
        intersection_values.append(intersection)

    return intersection_values

In [ ]:
def normalize_intersection_values(intersection_values):
    return np.array(intersection_values) / np.sum(intersection_values)

In [ ]:
import matplotlib.pyplot as plt

def plot_normalized_values(intersection_values, normalized_intersection_values):
    # Create subplots in a 1x2 grid
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))
    
    # Plot line graph for intersection_values
    axes[0].plot(intersection_values, label='Intersection Values')
    axes[0].set(xlabel='Frame Number', ylabel='Intersection Value', title='Intersection Values over Time')
    axes[0].legend()
    
    # Plot line graph for normalized_intersection_values
    axes[1].plot(normalized_intersection_values, label='Normalized Values', color='orange')
    axes[1].set(xlabel='Frame Number', ylabel='Normalized Value', title='Normalized Values over Time')
    axes[1].legend()
    
    # Adjust layout
    plt.tight_layout()
    plt.show()


video_path = 'Dataset/DatasetB.avi'
intersection_values = calculate_frame_intersection(video_path)
normalized_intersection_values = normalize_intersection_values(intersection_values)
plot_normalized_values(intersection_values, normalized_intersection_values)

In [ ]:
# b) Function to compute histogram intersection and normalization
def histogram_intersection(hist1, hist2):
    intersection = np.sum(np.minimum(hist1, hist2))
    normalization = intersection / np.sum(hist1)
    return intersection, normalization

In [ ]:
def compute_histogram(image):
    histogram = cv2.calcHist([image], [0, 1, 2], None, [256, 256, 256], [0, 256, 0, 256, 0, 256])
    histogram = histogram / histogram.sum()
    return histogram.flatten()

In [ ]:
def process_video_two_frames(lst):

    frame1 = lst[0]
    frame2 = lst[1]

    # # Convert frames to BGR if not already
    # if len(frame1.shape) == 2:
    #     frame1 = cv2.cvtColor(frame1, cv2.COLOR_GRAY2BGR)
    # if len(frame2.shape) == 2:
    #     frame2 = cv2.cvtColor(frame2, cv2.COLOR_GRAY2BGR)

    # Compute histogram for the current frames
    hist1 = cv2.calcHist(frame1, [0], None, [256], [0, 256])
    hist2 = cv2.calcHist(frame2, [0], None, [256], [0, 256])

    # Compute intersection and normalization
    intersection = calculate_histogram_intersection(hist1, hist2)
    normalization = normalize_intersection_values(intersection)

    # Plot intersection and normalized values
    plot_normalized_values(intersection, normalization)

In [ ]:
lst = frame_lst[8:10]
process_video_two_frames(lst)

### Texture Classification. (Use Datasets A and B)
The Local Binary Pattern (LBP) operator describes the surroundings of a pixel by generating a bit-code 
from the binary derivatives of a pixel. 
#### a) Write a function that divides a greyscale image into equally sized non-overlapping windows and returns the feature descriptor for each window as distribution of LBP codes. For each pixel in the window, compare the pixel to each of its 8 neighbours. Convert the resulting bit-codes (base 2) to decimals (base 10 numbers) and compute their histogram over the window. Normalize the histogram (which is now a feature descriptor representing the window). Show in the report the resulting images.

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, Image
import io

In [ ]:
def compute_lbp_histogram(window):
    """
    Compute the LBP histogram for a given window using vectorized operations.
    """
    # Calculate LBP codes for each pixel in the window
    lbp_codes = (window > np.roll(window, 1, axis=0)).astype(np.uint8)
    lbp_codes = 2**7 * lbp_codes + (window > np.roll(window, 1, axis=(0, 1))).astype(np.uint8)
    lbp_codes = 2**6 * lbp_codes + (window > np.roll(window, 0, axis=1)).astype(np.uint8)
    lbp_codes = 2**5 * lbp_codes + (window > np.roll(window, -1, axis=(0, 1))).astype(np.uint8)
    lbp_codes = 2**4 * lbp_codes + (window > np.roll(window, -1, axis=0)).astype(np.uint8)
    lbp_codes = 2**3 * lbp_codes + (window > np.roll(window, -1, axis=(1, 0))).astype(np.uint8)
    lbp_codes = 2**2 * lbp_codes + (window > np.roll(window, 0, axis=1)).astype(np.uint8)
    lbp_codes = 2**1 * lbp_codes + (window > np.roll(window, 1, axis=(1, 0))).astype(np.uint8)
    lbp_codes = lbp_codes + (window > window.mean()).astype(np.uint8)

    # Convert to decimal values
    decimal_values = np.packbits(lbp_codes.reshape(-1, 8)[:, ::-1], axis=1)

    # Compute histogram
    histogram, _ = np.histogram(decimal_values, bins=range(256), density=True)

    return histogram


In [ ]:
def extract_lbp_features(image, window_size):
    """
    Extract LBP features for the given greyscale image.
    """
    rows, cols = image.shape
    lbp_features = []

    for i in range(0, rows, window_size):
        for j in range(0, cols, window_size):
            window = image[i:i+window_size, j:j+window_size]

            # Check if the window size is smaller than the specified size
            if window.shape[0] < window_size or window.shape[1] < window_size:
                continue

            histogram = compute_lbp_histogram(window)
            lbp_features.append(histogram)

    return np.array(lbp_features)


In [ ]:
# Load greyscale image (replace 'image_path' with the actual file path)
image_path = '/home/jovyan/Computer Vision/Dataset/DatasetA/face-1.jpg'
gray_image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

# Set the window size
window_size = 16

# Extract LBP features from the greyscale image
lbp_features = extract_lbp_features(gray_image, window_size)

# Plot the histogram for all windows
plt.figure()
for i in range(lbp_features.shape[0]):
    plt.plot(lbp_features[i])

plt.title('LBP Histograms for All Windows')
plt.xlabel('Decimal Value')
plt.ylabel('Frequency')
plt.legend()
plt.show()


#### b) Come up with a descriptor that represents the whole image as consisting of multiple windows. For example, you could combine several local descriptions into a global description by concatenation.Discuss in the report alternative approaches. Using the global descriptor you created, implement a classification process that separates the images in the dataset into two categories: face images and non-face images (for example, you could use histogram similarities). Comment the results in the report. Is the global descriptor able to represent whole images of different types (e.g. faces vs. cars)? Identify problems (if any), discuss them in the report and suggest possible solutions

In [ ]:
def compute_lbp_histogram(window):
    """
    Compute the LBP histogram for a given window using vectorized operations.
    """
    # Calculate LBP codes for each pixel in the window
    lbp_codes = (window > np.roll(window, 1, axis=0)).astype(np.uint8)
    lbp_codes = 2**7 * lbp_codes + (window > np.roll(window, 1, axis=(0, 1))).astype(np.uint8)
    lbp_codes = 2**6 * lbp_codes + (window > np.roll(window, 0, axis=1)).astype(np.uint8)
    lbp_codes = 2**5 * lbp_codes + (window > np.roll(window, -1, axis=(0, 1))).astype(np.uint8)
    lbp_codes = 2**4 * lbp_codes + (window > np.roll(window, -1, axis=0)).astype(np.uint8)
    lbp_codes = 2**3 * lbp_codes + (window > np.roll(window, -1, axis=(1, 0))).astype(np.uint8)
    lbp_codes = 2**2 * lbp_codes + (window > np.roll(window, 0, axis=1)).astype(np.uint8)
    lbp_codes = 2**1 * lbp_codes + (window > np.roll(window, 1, axis=(1, 0))).astype(np.uint8)
    lbp_codes = lbp_codes + (window > window.mean()).astype(np.uint8)

    # Convert to decimal values
    decimal_values = np.packbits(lbp_codes.reshape(-1, 8)[:, ::-1], axis=1)

    # Compute histogram
    histogram, _ = np.histogram(decimal_values, bins=range(256), density=True)

    return histogram



def extract_lbp_features(image, window_size):
    """
    Extract LBP features for the given greyscale image.
    """
    rows, cols = image.shape
    lbp_features = []

    for i in range(0, rows, window_size):
        for j in range(0, cols, window_size):
            window = image[i:i+window_size, j:j+window_size]

            # Check if the window size is smaller than the specified size
            if window.shape[0] < window_size or window.shape[1] < window_size:
                continue

            histogram = compute_lbp_histogram(window)
            lbp_features.append(histogram)

    return np.array(lbp_features)

def create_global_descriptor(local_descriptors):
    """
    Create a global descriptor based on local descriptors.
    """
    # Concatenate local descriptors to create a global descriptor
    global_descriptor = np.concatenate(local_descriptors, axis=0)

    # Normalize the global descriptor
    global_descriptor /= np.linalg.norm(global_descriptor)

    return global_descriptor

# Example usage:
# Load greyscale image for face and non-face (replace 'image_path' with the actual file paths)
image_path_face = '/home/jovyan/Computer Vision/Dataset/DatasetA/face-1.jpg'
image_path_non_face = '/home/jovyan/Computer Vision/Dataset/DatasetA/car-1.jpg'

# Set the window size
window_size = 16

# Extract LBP features from the greyscale images
lbp_features_face = extract_lbp_features(cv2.imread(image_path_face, cv2.IMREAD_GRAYSCALE), window_size)
lbp_features_non_face = extract_lbp_features(cv2.imread(image_path_non_face, cv2.IMREAD_GRAYSCALE), window_size)

# Create global descriptors for face and non-face images
global_descriptor_face = create_global_descriptor(lbp_features_face)
global_descriptor_non_face = create_global_descriptor(lbp_features_non_face)

# Load an image to be classified (replace 'test_image_path' with the actual file path)
test_image_path = '/home/jovyan/Computer Vision/Dataset/DatasetA/car-2.jpg'
lbp_features_test = extract_lbp_features(cv2.imread(test_image_path, cv2.IMREAD_GRAYSCALE), window_size)
global_descriptor_test = create_global_descriptor(lbp_features_test)

# Simple classification process
# Assume a threshold value for classification
threshold = 0.7

# Calculate similarity for face
similarity_face = np.dot(global_descriptor_test, global_descriptor_face) / (np.linalg.norm(global_descriptor_test) * np.linalg.norm(global_descriptor_face))

# Calculate similarity for non-face
similarity_non_face = np.dot(global_descriptor_test, global_descriptor_non_face) / (np.linalg.norm(global_descriptor_test) * np.linalg.norm(global_descriptor_non_face))

# Classification process is based on histogram similarities
if similarity_face > similarity_non_face and similarity_face > threshold:
    print("Image: Face")
elif similarity_non_face > similarity_face and similarity_non_face > threshold:
    print("Image: Non-Face")
else:
    print("Image: Unknown")



#### c) Decrease the window size and perform classification again. Comment the results in the report. 

In [ ]:
import cv2
import numpy as np

def extract_lbp_features(image, window_size):
    """
    Extract LBP features for the given image.
    """
    # Convert the image to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    rows, cols = gray_image.shape
    lbp_features = []

    for i in range(0, rows, window_size):
        for j in range(0, cols, window_size):
            window = gray_image[i:i+window_size, j:j+window_size]

            # Check if the window size is smaller than the specified size
            if window.shape[0] < window_size or window.shape[1] < window_size:
                continue

            # Compute LBP histogram for the window
            histogram = compute_lbp_histogram(window)
            lbp_features.append(histogram)

    return np.array(lbp_features)

# Load face image
face_image = cv2.imread(image_path_face)
if face_image is None:
    raise ValueError(f"Failed to load image: {image_path_face}")

# Load non-face image
non_face_image = cv2.imread(image_path_non_face)
if non_face_image is None:
    raise ValueError(f"Failed to load image: {image_path_non_face}")

# Load an image to be classified
test_image = cv2.imread(test_image_path)
if test_image is None:
    raise ValueError(f"Failed to load image: {test_image_path}")

# Set the window size
window_size = 32
window_size_small = 12

# Extract LBP features from the color images
lbp_features_face = extract_lbp_features(face_image, window_size)
lbp_features_non_face = extract_lbp_features(non_face_image, window_size)

lbp_features_face_small = extract_lbp_features(face_image, window_size_small)
lbp_features_non_face_small = extract_lbp_features(non_face_image, window_size_small)

# Create global descriptors for face and non-face images
global_descriptor_face = create_global_descriptor(lbp_features_face)
global_descriptor_non_face = create_global_descriptor(lbp_features_non_face)

global_descriptor_face_small = create_global_descriptor(lbp_features_face_small)
global_descriptor_non_face_small = create_global_descriptor(lbp_features_non_face_small)

# Extract LBP features from the test image
lbp_features_test = extract_lbp_features(test_image, window_size)
lbp_features_test_small = extract_lbp_features(test_image, window_size_small)

# Create global descriptors for the test image
global_descriptor_test = create_global_descriptor(lbp_features_test)
global_descriptor_test_small = create_global_descriptor(lbp_features_test_small)

# Assume a threshold value for classification
threshold = 0.4

# Calculate similarity for face
similarity_face = np.dot(global_descriptor_test, global_descriptor_face) / (np.linalg.norm(global_descriptor_test) * np.linalg.norm(global_descriptor_face))

# Calculate similarity for non-face
similarity_non_face = np.dot(global_descriptor_test, global_descriptor_non_face) / (np.linalg.norm(global_descriptor_test) * np.linalg.norm(global_descriptor_non_face))

# Classification process is based on histogram similarities
if similarity_face > threshold:
    print("Image: Face")
else:
    print("Image: Non-Face")

# Perform the classification with the decreased window size
# Calculate similarity for face (small window size)
similarity_face_small = np.dot(global_descriptor_test_small, global_descriptor_face_small) / (np.linalg.norm(global_descriptor_test_small) * np.linalg.norm(global_descriptor_face_small))

# Calculate similarity for non-face (small window size)
similarity_non_face_small = np.dot(global_descriptor_test_small, global_descriptor_non_face_small) / (np.linalg.norm(global_descriptor_test_small) * np.linalg.norm(global_descriptor_non_face_small))

# Classification process is based on histogram similarities with the decreased window size
if similarity_face_small > threshold:
    print("Image (small window size): Face")
else:
    print("Image (small window size): Non-Face")

#### d) Increase the window size and perform classification again. Comment the results in the report.¶

In [ ]:
import cv2
import numpy as np

def extract_lbp_features(image, window_size):
    """
    Extract LBP features for the given image.
    """
    # Convert the image to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    rows, cols = gray_image.shape
    lbp_features = []

    for i in range(0, rows, window_size):
        for j in range(0, cols, window_size):
            window = gray_image[i:i+window_size, j:j+window_size]

            # Check if the window size is smaller than the specified size
            if window.shape[0] < window_size or window.shape[1] < window_size:
                continue

            # Compute LBP histogram for the window
            histogram = compute_lbp_histogram(window)
            lbp_features.append(histogram)

    return np.array(lbp_features)

# Load face image
face_image = cv2.imread(image_path_face)
if face_image is None:
    raise ValueError(f"Failed to load image: {image_path_face}")

# Load non-face image
non_face_image = cv2.imread(image_path_non_face)
if non_face_image is None:
    raise ValueError(f"Failed to load image: {image_path_non_face}")

# Load an image to be classified
test_image = cv2.imread(test_image_path)
if test_image is None:
    raise ValueError(f"Failed to load image: {test_image_path}")

# Set the window size
window_size = 32
window_size_large = 12

# Extract LBP features from the color images
lbp_features_face = extract_lbp_features(face_image, window_size)
lbp_features_non_face = extract_lbp_features(non_face_image, window_size)

lbp_features_face_large = extract_lbp_features(face_image, window_size_large)
lbp_features_non_face_large = extract_lbp_features(non_face_image, window_size_large)

# Create global descriptors for face and non-face images
global_descriptor_face = create_global_descriptor(lbp_features_face)
global_descriptor_non_face = create_global_descriptor(lbp_features_non_face)

global_descriptor_face_large = create_global_descriptor(lbp_features_face_large)
global_descriptor_non_face_large = create_global_descriptor(lbp_features_non_face_large)

# Extract LBP features from the test image
lbp_features_test = extract_lbp_features(test_image, window_size)
lbp_features_test_large = extract_lbp_features(test_image, window_size_large)

# Create global descriptors for the test image
global_descriptor_test = create_global_descriptor(lbp_features_test)
global_descriptor_test_large = create_global_descriptor(lbp_features_test_large)

# Assume a threshold value for classification
threshold = 0.4

# Calculate similarity for face
similarity_face = np.dot(global_descriptor_test, global_descriptor_face) / (np.linalg.norm(global_descriptor_test) * np.linalg.norm(global_descriptor_face))

# Calculate similarity for non-face
similarity_non_face = np.dot(global_descriptor_test, global_descriptor_non_face) / (np.linalg.norm(global_descriptor_test) * np.linalg.norm(global_descriptor_non_face))

# Classification process is based on histogram similarities
if similarity_face > threshold:
    print("Image: Face")
else:
    print("Image: Non-Face")

# Perform the classification with the decreased window size
# Calculate similarity for face (large window size)
similarity_face_large = np.dot(global_descriptor_test_large, global_descriptor_face_large) / (np.linalg.norm(global_descriptor_test_large) * np.linalg.norm(global_descriptor_face_large))

# Calculate similarity for non-face (large window size)
similarity_non_face_large = np.dot(global_descriptor_test_large, global_descriptor_non_face_large) / (np.linalg.norm(global_descriptor_test_large) * np.linalg.norm(global_descriptor_non_face_large))

# Classification process is based on histogram similarities with the decreased window size
if similarity_face_large > threshold:
    print("Image (large window size): Face")
else:
    print("Image (large window size): Non-Face")

### 5) Object Counting. (Use Dataset C)
Moving objects captured by fixed cameras are the focus of several computer vision applications.
#### a) Write a function that performs pixel-by-pixel frame differencing using, as reference frame, the first frame of an image sequence. Apply a classification threshold and save the results.

In [ ]:


def frame_difference(video_path, output_path, threshold=30):
    # Open the video file
    cap = cv2.VideoCapture(video_path)

    # Read the first frame as the reference frame
    ret, reference_frame = cap.read()

    # Convert the reference frame to grayscale
    reference_gray = cv2.cvtColor(reference_frame, cv2.COLOR_BGR2GRAY)

    # List to store frames
    frames = []

    while True:
        # Read a new frame
        ret, frame = cap.read()
        if not ret:
            break

        # Convert the frame to grayscale
        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Compute the absolute difference between the current frame and the reference frame
        frame_diff = cv2.absdiff(reference_gray, frame_gray)

        # Apply a threshold to classify pixels as moving or static
        _, thresholded = cv2.threshold(frame_diff, threshold, 255, cv2.THRESH_BINARY)

        # Save the result
        frames.append(thresholded)

    # Release resources
    cap.release()

    # Display frames in Jupyter notebook
    for i, frame in enumerate(frames):
        image_data = cv2.imencode('.png', frame)[1].tobytes()
        display(Image(data=image_data, format='png'))

# Example usage
video_path = 'Dataset/DatasetC.avi'
output_path = 'Dataset/DatasetC_result.avi'
frame_difference(video_path, output_path, threshold=30)


In [ ]:
import cv2

def frame_difference(video_path, output_path, threshold=30):
    # Open the video file
    cap = cv2.VideoCapture(video_path)

    # Read the first frame as the initial reference frame
    ret, reference_frame = cap.read()

    # Convert the initial reference frame to grayscale
    reference_gray = cv2.cvtColor(reference_frame, cv2.COLOR_BGR2GRAY)

    # Define the codec and create VideoWriter object for saving the result
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(output_path, fourcc, 20.0, (reference_frame.shape[1], reference_frame.shape[0]))

    while True:
        # Read a new frame
        ret, frame = cap.read()
        if not ret:
            break

        # Convert the frame to grayscale
        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Compute the absolute difference between the current frame and the previous frame
        frame_diff = cv2.absdiff(reference_gray, frame_gray)

        # Apply a threshold to classify pixels as moving or static
        _, thresholded = cv2.threshold(frame_diff, threshold, 255, cv2.THRESH_BINARY)

        # Save the result
        out.write(thresholded)

        # Update the reference frame for the next iteration
        reference_gray = frame_gray

    # Release resources
    cap.release()
    
    out.release()

# Example usage
video_path = 'Dataset/DatasetC.avi'
output_path = 'Dataset/DatasetC_result_using_previous_frame.avi'
frame_difference(video_path, output_path, threshold=30)


#### b) Repeat the exercise using the previous frame as reference frame (use frame It-1 as reference frame for frame It, for each t). Comment the results in the report

In [ ]:
import cv2
from IPython.display import display, Image
import io

def frame_difference(video_path, threshold=30):
    # Open the video file
    cap = cv2.VideoCapture(video_path)

    # Read the first frame as the initial reference frame
    ret, reference_frame = cap.read()

    # Convert the initial reference frame to grayscale
    reference_gray = cv2.cvtColor(reference_frame, cv2.COLOR_BGR2GRAY)

    # List to store frames
    frames = []

    while True:
        # Read a new frame
        ret, frame = cap.read()
        if not ret:
            break

        # Convert the frame to grayscale
        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Compute the absolute difference between the current frame and the previous frame
        frame_diff = cv2.absdiff(reference_gray, frame_gray)

        # Apply a threshold to classify pixels as moving or static
        _, thresholded = cv2.threshold(frame_diff, threshold, 255, cv2.THRESH_BINARY)

        # Display the result (optional)
        image_data = cv2.imencode('.png', thresholded)[1].tobytes()
        display(Image(data=image_data, format='png'))

        # Update the reference frame for the next iteration
        reference_gray = frame_gray

# Example usage
video_path = 'Dataset/DatasetC.avi'
frame_difference(video_path, threshold=30)


#### c) Write a function that generates a reference frame (background) for the sequence using for example frame differencing and a weighted temporal averaging algorithm.

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

def generate_reference_frame(video_path, alpha=0.01, threshold=30):
    # Open the video file
    cap = cv2.VideoCapture(video_path)

    # Read the first frame as the initial reference frame
    ret, reference_frame = cap.read()

    # Convert the initial reference frame to grayscale
    reference_gray = cv2.cvtColor(reference_frame, cv2.COLOR_BGR2GRAY)

    # Initialize the accumulated frame for weighted temporal averaging
    accumulated_frame = np.float32(reference_gray)

    # Process each frame in the video sequence
    while True:
        # Read a new frame
        ret, frame = cap.read()
        if not ret:
            break


        # Convert the frame to grayscale
        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Compute the absolute difference between the current frame and the accumulated frame
        frame_diff = cv2.absdiff(accumulated_frame.astype(np.uint8), frame_gray)

        # Apply a binary threshold to classify pixels as moving or static
        _, thresholded = cv2.threshold(frame_diff, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)


        # Weighted temporal averaging algorithm
        cv2.accumulateWeighted(frame_gray, accumulated_frame, alpha)

    # Release resources
    cap.release()

    # Convert the accumulated frame to uint8
    reference_frame_result = cv2.convertScaleAbs(accumulated_frame)

    return reference_frame_result

def display_image(image, title="Image"):
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.title(title)
    plt.axis('off')
    plt.show()

# Example usage
video_path = 'Dataset/DatasetC.mpg'
reference_frame = generate_reference_frame(video_path, alpha=0.01, threshold=50)



# Display the generated reference frame
display_image(reference_frame, title='Generated Reference Frame')


#### d) Write a function that counts the number of moving objects in each frame of a sequence. Generate a bar plot that visualizes the number of objects for each frame of the whole sequence. Discuss in the report the implemented solution, including advantages and disadvantages.

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

def count_moving_objects(video_path):
    cap = cv2.VideoCapture(video_path)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    object_counts = []

    # Create background subtractor
    bg_subtractor = cv2.createBackgroundSubtractorMOG2()

    for _ in range(frame_count):
        _, frame = cap.read()

        # Apply background subtraction
        fg_mask = bg_subtractor.apply(frame)

        # Threshold the mask
        _, thresh = cv2.threshold(fg_mask, 9, 255, cv2.THRESH_BINARY)

        # Find contours of moving objects
        contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # Count the number of moving objects in the current frame
        object_count = len(contours)
        object_counts.append(object_count)

    cap.release()
    return object_counts

def plot_object_counts(object_counts):
    # Increase the figure size
    plt.figure(figsize=(12, 6))

    plt.bar(range(1, len(object_counts) + 1), object_counts)
    plt.xlabel('Frame Number')
    plt.ylabel('Number of Moving Objects')
    plt.title('Number of Moving Objects in Each Frame')
    plt.show()

# Example usage
video_path = 'Dataset/DatasetC.mpg'
object_counts = count_moving_objects(video_path)
plot_object_counts(object_counts)